In [ ]:
#For Webscraping
from selenium import webdriver

#To Pull Stock Data
import pandas_datareader.data as web

#Other useful modules
import time, datetime
import pandas as pd

#For data visualization
import plotly
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
#Path to ChromeWebDriver
PATH = r'Path\chromedriver.exe'
#Ticker
ticker = 'GOOG'

In [ ]:
driver = webdriver.Chrome(PATH)

#Start Website
driver.minimize_window()
driver.get('https://finance.yahoo.com/quote/' + ticker + '/financials?p=' + ticker)

#Scraping data for Income Statement
expand = driver.find_element_by_xpath('//div[@id="Col1-1-Financials-Proxy"]//div[@class="Mb(10px)"]//button').click()
time.sleep(5)
IScolumnTitles = [x.text for x in driver.find_elements_by_xpath('//div[@class="D(tbr) C($primaryColor)"]//div//span')]
ISfinancialNumbers = [x.text for x in driver.find_elements_by_xpath('//div[@class="D(tbrg)"]//div[@class="rw-expnded"]//div[@data-test="fin-col"]')]

#Removing all commas from strings
ISfinancialNumbers = [x.replace(",", "") for x in ISfinancialNumbers]

ISrowTitles = [x.text for x in driver.find_elements_by_xpath('//div[@class="rw-expnded"]//span[@class="Va(m)"]')]

balance_tab = driver.find_element_by_xpath('//div[@class="D(ib) Pos(r)"]//a[@role="tab"]').click() #So the next page can start loading
column_count = len(IScolumnTitles[1:])
ISNums = [ISfinancialNumbers[i:i+column_count] for i in range(0, len(ISfinancialNumbers), column_count)]

income_statement = pd.DataFrame(ISNums, ISrowTitles, IScolumnTitles[1:])

time.sleep(5)

#Scraping data for Balance Sheet
expand = driver.find_element_by_xpath('//div[@id="Col1-1-Financials-Proxy"]//div[@class="Mb(10px)"]//button').click()
BScolumnTitles = [x.text for x in driver.find_elements_by_xpath('//div[@class="D(tbr) C($primaryColor)"]//div//span')]
BSfinancialNumbers = [x.text for x in driver.find_elements_by_xpath('//div[@class="D(tbrg)"]//div[@class="rw-expnded"]//div[@data-test="fin-col"]')]

#Removing all commas from strings
BSfinancialNumbers = [x.replace(",", "") for x in BSfinancialNumbers]

BSrowTitles = [x.text for x in driver.find_elements_by_xpath('//div[@class="rw-expnded"]//span[@class="Va(m)"]')]

cf_tab = driver.find_element_by_xpath('(//a[@class="Mend(10px) P(0px) M(0px) C($linkColor) C($primaryColor):h Bd(0px)"])[2]').click()
column_count = len(BScolumnTitles[1:])
BSNums = [BSfinancialNumbers[i:i+column_count] for i in range(0, len(BSfinancialNumbers), column_count)]

balance_sheet = pd.DataFrame(BSNums, BSrowTitles, BScolumnTitles[1:])

driver.close()
time.sleep(5)

In [ ]:
driver = webdriver.Chrome(PATH)

#Closed previous session and open a new one due to load page performance. 
driver.minimize_window()
driver.get('https://finance.yahoo.com/quote/' + ticker + '/cash-flow?p=' + ticker)

#Cash Flow
expand = driver.find_element_by_xpath('//div[@id="Col1-1-Financials-Proxy"]//div[@class="Mb(10px)"]//button').click()
time.sleep(5)
CFcolumnTitles = [x.text for x in driver.find_elements_by_xpath('//div[@class="D(tbr) C($primaryColor)"]//div//span')]
CFfinancialNumbers = [x.text for x in driver.find_elements_by_xpath('//div[@class="D(tbrg)"]//div[@class="rw-expnded"]//div[@data-test="fin-col"]')]

#Removing all commas from strings
CFfinancialNumbers = [x.replace(",", "") for x in CFfinancialNumbers]

CFrowTitles = [x.text for x in driver.find_elements_by_xpath('//div[@class="rw-expnded"]//span[@class="Va(m)"]')]

column_count = len(CFcolumnTitles[1:])
CFNums = [CFfinancialNumbers[i:i+column_count] for i in range(0, len(CFfinancialNumbers), column_count)]

cash_flow = pd.DataFrame(CFNums, CFrowTitles, CFcolumnTitles[1:])

driver.close()

In [ ]:
#Currently all data is a string. Need to convert it to a number. 
income_statement = income_statement.replace("-", 0).astype(float)
balance_sheet = balance_sheet.replace("-", 0).astype(float)
cash_flow = cash_flow.replace("-", 0).astype(float)

In [ ]:
#Saving the financial data to a spreadsheet
with pd.ExcelWriter(ticker + ' Financial Data.xlsx') as writer:
    income_statement.to_excel(writer, sheet_name='Income Statement')
    balance_sheet.to_excel(writer, sheet_name='Balance Sheet')
    cash_flow.to_excel(writer, sheet_name='Cash Flow')

In [ ]:
#Start Date will be the beginning of the oldest year in the financial statements
dt = datetime.datetime.strptime(IScolumnTitles[-1], '%m/%d/%Y')
start = datetime.datetime(dt.year, 1, 1)

#End Date
end = datetime.date.today()


stock_data = web.DataReader(ticker, "av-daily", start=start, end=end, api_key='ALPHAVANTAGE_API_KEY')

#Also pulling the daily stock deltas
stock_delta = pd.DataFrame()
stock_delta['Change'] = stock_data['close'].pct_change()
stock_delta.index.rename('Date', inplace=True)
stock_delta['Change'] = stock_delta['Change'].apply(lambda x: x*100)

In [ ]:
stock_chart = go.Figure(data=[go.Candlestick(x=stock_data.index, open=stock_data['open'], high=stock_data['high'], low=stock_data['low'], close=stock_data['close'])])

stock_chart.update_layout(
title=ticker + ' Stock Historical Data',
yaxis_title='Stock Price')

In [ ]:
delta_graph = px.line(stock_delta, y="Change", title=ticker + ' Daily Delta in %')
delta_graph.show()

In [ ]:
stock_delta.describe()

In [ ]:
IS = income_statement.loc[['Total Revenue', 'Gross Profit', 'Net Income']].T.reset_index()[::-1]
income_statement_chart = px.line(IS, x=IS['index'], y=IS.columns)
income_statement_chart.update_layout(
title=ticker,
yaxis_title='USD in Thousands',
xaxis_title='Date',
legend_title_text='Breakdown')